In [ ]:
import pandas as pd
from collections import defaultdict
import os
import math
import glob
import json

In [ ]:
import pandas as pd
from collections import defaultdict
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import os
import math
import itertools

In [ ]:
import swifter

In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
from tqdm import tqdm

In [ ]:
tqdm.pandas()

In [ ]:
ALL_SCHOOLS_DATA_DIR = '/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/all_schools_data/csvs/'


In [ ]:
all_files = glob.glob(os.path.join(ALL_SCHOOLS_DATA_DIR, "*.csv"))


In [ ]:
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
df['inv_unique_id'] = df['provider_and_id'].astype(str) + '_' +  df['involve_id'].astype(str) + '_' + df['accident_year'].astype(str)


In [ ]:
df['acc_unique_id'] = df['provider_and_id'].astype(str) + '_' + df['accident_year'].astype(str)


In [ ]:
df['vehicle_or_pedastrian'] = df.apply(lambda x: x['involve_vehicle_type_hebrew'] if x['injured_type_hebrew'] != 'הולך רגל' else  x['injured_type_hebrew'], axis=1)


In [ ]:
df['accident_timestamp'] = pd.to_datetime(df['accident_timestamp'])

In [ ]:
months_dict = {1: 'ינואר', 
               2: 'פברואר',
               3: 'מרס',
               4: 'אפריל',
               5: 'מאי',
               6: 'יוני',
               7: 'יולי',
               8: 'אוגוסט',
               9: 'ספטמבר',
               10: 'אוקטובר',
               11: 'נובמבר',
               12: 'דצמבר'}

In [ ]:
df['accident_month_hebrew'] = df['accident_month'].apply(lambda m: months_dict.get(m))

### validate 5 years

In [ ]:
assert(round(((df['accident_timestamp'].max() - df['accident_timestamp'].min()).days / 365),0) == 5)

In [ ]:
df.to_csv('all_injured_per_schools.csv')

In [ ]:
df_unique_injured = df.drop_duplicates('inv_unique_id').copy()

In [ ]:
df_unique_injured.to_csv('all_unique_injured.csv')

In [ ]:
df_injured_in_all_schools = df.groupby(['injury_severity'])["inv_unique_id"].nunique()

In [ ]:
df_injured_in_all_schools.to_csv('df_injured_in_all_schools.csv', index=False)

In [ ]:
df_injured_in_all_schools_per_v_or_p = df.groupby(['injury_severity', 'vehicle_or_pedastrian'])["inv_unique_id"].nunique()


In [ ]:
df_injured_in_all_schools_per_p_c = df.groupby(['injury_severity', 'provider_code'])["inv_unique_id"].nunique()


In [ ]:
df_injured_in_all_schools_per_v_or_p.to_csv('df_injured_in_all_schools_per_type.csv', index=False)

In [ ]:
df_injured_in_all_schools_per_p_c.to_csv('df_injured_in_all_schools_per_p_c.csv', index=False)

In [ ]:
killed_weight = 6600/7581
severe_weight = 956/7581
light_weight = 25/7581

### calc per school

In [ ]:
df_total_injured_schools = (
    df.groupby(
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
            "injury_severity"
        ]
    )
    .size()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "injury_severity",
            "injured_count",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        ],
    ]
)
df_total_injured_schools = df_total_injured_schools.set_index(
    [
        "school_yishuv_name",
        "school_id",
        "school_name",
        "institution_type",
        "school_anyway_link",
        "school_longitude",
        "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        "injury_severity",

    ]
).unstack(-1)
df_total_injured_schools.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)

df_total_injured_schools.loc[:, (slice("injured_count"), slice(None))] = df_total_injured_schools.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured_schools["total_injured_count"] = (
    df_total_injured_schools.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

In [ ]:
df_total_injured_schools.columns = ['killed_count', 'severe_injured_count', 'light_injured_count', 'total_injured_count']
                                                       

In [ ]:
df_total_injured_per_veh_or_ped = (
    df.groupby(
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
            "vehicle_or_pedastrian"
        ]
    )
    .size()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "vehicle_or_pedastrian",
            "injured_count",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        ],
    ]
)
df_total_injured_per_veh_or_ped = df_total_injured_per_veh_or_ped.set_index(
    [
        "school_yishuv_name",
        "school_id",
        "school_name",
        "institution_type",
        "school_anyway_link",
        "school_longitude",
        "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        "vehicle_or_pedastrian",

    ]
).unstack(-1)

df_total_injured_per_veh_or_ped.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)


In [ ]:
df_total_injured_per_veh_or_ped.columns = ['bike', 'electric_bike', 'pedastrians', 'electric_korki']

In [ ]:
df_total_accidents_schools = (
    df.groupby(
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
        ]
    )["acc_unique_id"]
    .nunique()
    .reset_index(name="accidents_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "school_id",
            "school_name",
            "institution_type",
            "school_anyway_link",
            "school_longitude",
            "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6",
            "accidents_count"
        ],
    ]
)

df_total_accidents_schools = df_total_accidents_schools.set_index(
[
        "school_yishuv_name",
        "school_id",
        "school_name",
        "institution_type",
        "school_anyway_link",
        "school_longitude",
        "school_latitude",
            "WKT Polygon",
            "WKT Point_12",
            "WKT Point_6"])

In [ ]:
df_total = pd.merge(df_total_injured_schools,
                     df_total_accidents_schools,
                     left_index=True,
                     right_index=True,
                     how='left')

In [ ]:
df_total = pd.merge(df_total,
                     df_total_injured_per_veh_or_ped,
                     left_index=True,
                     right_index=True,
                     how='left')

In [ ]:
def calc_prat_score(x):
    prat_score = (x["killed_count"] + x["severe_injured_count"] + x["light_injured_count"]) * \
   (x["killed_count"] * killed_weight + \
    x["severe_injured_count"]  * severe_weight + \
    x["light_injured_count"]  * light_weight)
    return prat_score


In [ ]:
def calc_heuristic_score(x):
    h_score = x["killed_count"] * 8 + \
    x["severe_injured_count"]  * 5 + \
    x["light_injured_count"]  * 1
    return h_score

In [ ]:
df_total['prat_score'] = df_total.apply(lambda x: calc_prat_score(x), axis=1)

In [ ]:
df_total['h_score'] = df_total.apply(lambda x: calc_heuristic_score(x), axis=1)

In [ ]:
df_total = df_total.sort_values('prat_score', ascending=False)

In [ ]:
df_total.to_csv('scores_per_school_both_tik1_and_tik3.csv')

### calc per yisuv

In [ ]:
df_total_injured = (
    df.groupby(
        [
            "school_yishuv_name",
            "injury_severity"
        ]
    )["inv_unique_id"]
    .nunique()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "injury_severity",
            "injured_count",
        ],
    ]
)
df_total_injured = df_total_injured.set_index(
    [
        "school_yishuv_name",
        "injury_severity",

    ]
).unstack(-1)
df_total_injured.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)


df_total_injured.loc[:, (slice("injured_count"), slice(None))] = df_total_injured.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured["total_injured_count"] = (
    df_total_injured.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

df_total_accidents_yishuvs = (
    df.groupby(
        [
            "school_yishuv_name",
        ]
    )["acc_unique_id"]
    .nunique()
    .reset_index(name="accidents_count")
    .loc[
        :,
        [
            "school_yishuv_name",
            "accidents_count"
        ],
    ]
)

df_total_accidents_yishuvs = df_total_accidents_yishuvs.set_index(
[
        "school_yishuv_name"])

df_total_yishuvs = pd.merge(df_total_injured,
                     df_total_accidents_yishuvs,
                     left_index=True,
                     right_index=True,
                     how='left')

df_total_yishuvs.columns = ['killed_count', 'severe_injured_count', 'light_injured_count', 'total_injured_count', 'accidents_count']

df_total_yishuvs['prat_score'] = df_total_yishuvs.apply(lambda x: calc_prat_score(x), axis=1)

df_total_yishuvs['h_score'] = df_total_yishuvs.apply(lambda x: calc_heuristic_score(x), axis=1)

df_total_yishuvs = df_total_yishuvs.sort_values('prat_score', ascending=False)

df_total_yishuvs.to_csv('scores_per_yishuv_both_tik1_and_tik3.csv')


In [ ]:
df_total_yishuvs.sum()

In [ ]:
top_20_yishuvs = df_total_yishuvs['prat_score'].sort_values(ascending=False).iloc[0:20]

### Choose Polygons

In [ ]:
def calc_dict_for_prat(df):
    killed = df.loc[df.loc[:,'injury_severity'] == 1].inv_unique_id.nunique()
    severe_injured = df.loc[df.loc[:,'injury_severity'] == 2].inv_unique_id.nunique()
    light_injured = df.loc[df.loc[:,'injury_severity'] == 3].inv_unique_id.nunique()
    total_accidents = df.acc_unique_id.nunique()
    
    final_dict = {"accidents_count": total_accidents,
                  "light_injured_count": light_injured,
                  "severe_injured_count": severe_injured,
                  "killed_count": killed}
    return final_dict


In [ ]:
top_20_yishuvs

In [ ]:
def get_final_poly_school_ids(yiushuv_name):
    print(yiushuv_name)
    df_yishuv_schools = df_total[df_total.index.get_level_values('school_yishuv_name').isin([yiushuv_name])].iloc[0:50]
    final_combs_with_scores = []
    comb_size = df_yishuv_schools.shape[0]
    if comb_size == 0:
        return []
    elif comb_size > 3:
        comb_size = 3
    else:
        pass
    all_school_ids = set(itertools.combinations(df_yishuv_schools.index.values, comb_size))
    all_school_ids = [list(item) for item in set(tuple(row) for row in all_school_ids)]
    for comb in tqdm(all_school_ids):
        school_ids = [comb[0][1], comb[1][1], comb[2][1]]
        df_curr = df.loc[df.school_id.isin(school_ids)]
        prat_dict = calc_dict_for_prat(df_curr)
        prat_score = calc_prat_score(prat_dict)
        final_combs_with_scores.append({"school_ids": school_ids,
                                        "prat_score": prat_score})
    school_ids_final = pd.DataFrame(final_combs_with_scores).sort_values('prat_score', ascending=False)['school_ids'].iloc[0]
    return school_ids_final


In [ ]:
top_20_yishuvs = top_20_yishuvs.to_frame().reset_index()

In [ ]:
poly_school_ids_final = top_20_yishuvs.swifter.apply(lambda x: get_final_poly_school_ids(x['school_yishuv_name']), axis=1)


In [ ]:
poly_school_ids_final = poly_school_ids_final.sum()

### create final tables

In [ ]:
df_total_no_idx = df_total.reset_index()

In [ ]:
def get_bounding_box(latitude, longitude, distance_in_km):
    latitude = math.radians(latitude)
    longitude = math.radians(longitude)

    radius = 6371
    # Radius of the parallel at given latitude
    parallel_radius = radius * math.cos(latitude)

    lat_min = latitude - distance_in_km / radius
    lat_max = latitude + distance_in_km / radius
    lon_min = longitude - distance_in_km / parallel_radius
    lon_max = longitude + distance_in_km / parallel_radius
    rad2deg = math.degrees

    return rad2deg(lat_min), rad2deg(lon_min), rad2deg(lat_max), rad2deg(lon_max)

In [ ]:
all_relevant_schools = []
for school_id in poly_school_ids_final:
    center_lat, center_lon = df_total_no_idx.loc[df_total_no_idx.school_id == school_id, ['school_latitude', 'school_longitude']].values[0]
    lat_min, lon_min, lat_max, lon_max = get_bounding_box(center_lat, center_lon, 0.5)
    baseX = lon_min
    baseY = lat_min
    distanceX = lon_max
    distanceY = lat_max

    poly = Polygon([(baseX, baseY), 
                    (baseX, distanceY),
                    (distanceX, distanceY),
                    (distanceX, baseY)])
    bnbr_schools = df_total_no_idx.drop_duplicates(['school_id','school_longitude', 'school_latitude']).loc[:,['school_id','school_longitude', 'school_latitude']].to_dict(orient='records')
    schools_in_1km_box = [r['school_id'] for r in bnbr_schools if poly.contains(Point(r['school_longitude'], r['school_latitude']))]
    all_relevant_schools += schools_in_1km_box


### polygons of final schools

In [ ]:
final_polygons_csv = df_total_no_idx.loc[df_total_no_idx.school_id.isin(poly_school_ids_final), ['WKT Polygon',
                                                                                                'school_yishuv_name',
                                                                                                'school_name',
                                                                                                'prat_score',
                                                                                                'killed_count',
                                                                                                'severe_injured_count',
                                                                                                'light_injured_count',
                                                                                                'total_injured_count',
                                                                                                'pedastrians',
                                                                                                'bike',
                                                                                                'electric_bike',
                                                                                                'electric_korki',
                                                                                                'accidents_count']].reset_index(drop=True)


In [ ]:
final_polygons_csv['WKT'] = final_polygons_csv['WKT Polygon'].apply(lambda x: x.replace(",", ", "))

In [ ]:
relevant_cols = ['WKT', 
                 'school_name',
                 'school_yishuv_name',
                 'prat_score',
                 'killed_count',
                 'severe_injured_count',
                 'light_injured_count',
                 'total_injured_count',
                 'pedastrians',
                 'bike',
                 'electric_bike',
                 'electric_korki',
                 'accidents_count']

In [ ]:
final_polygons_csv = final_polygons_csv.loc[:, relevant_cols]

In [ ]:
final_polygons_csv['school_name'] = 'מקבץ ' + final_polygons_csv['school_name']

In [ ]:
final_polygons_csv['prat_score'] = final_polygons_csv['prat_score'].round(0).astype(int)

In [ ]:
final_polygons_csv

In [ ]:
relevant_hebrew_cols = ['WKT',
                  'שם מקבץ',
                  'ישוב',
                  'דירוג',
                  'סה״כ הרוגים במקבץ',
                  'סה״כ פצועים קשה במקבץ',
                  'סה״כ פצועים קל במקבץ',
                  'סה״כ נפגעים והרוגים',
                  'סה״כ הולכי רגל נפגעים',
                  'סה״כ נפגעים רוכבי אופניים',
                  'סה״כ נפגעים רוכבי אופניים חשמלים',
                  'סה״כ נפגעים רוכבי קורקינט חשמלי',
                  'מספר תאונות']

In [ ]:
final_polygons_csv.columns = relevant_hebrew_cols

In [ ]:
final_polygons_csv.to_csv('final_polygons_csv.csv', index=False)

### final schools and injured csvs for maps

In [ ]:
final_schools_csv = df_total_no_idx.loc[df_total_no_idx.school_id.isin(all_relevant_schools), ['school_yishuv_name', 'school_name', 'school_longitude', 'school_latitude']]


In [ ]:
final_schools_csv.columns = ['ישוב' , 'שם מוסד', 'longitude', 'latitude']

In [ ]:
final_schools_csv.to_csv('final_schools_csv.csv', index=False)


In [ ]:
final_schools_csv

In [ ]:
final_injured_csv = df.loc[df.school_id.isin(all_relevant_schools), ['vehicle_or_pedastrian',  'injury_severity_hebrew', 'age_group_hebrew', 'accident_timestamp' ,'longitude', 'latitude', 'inv_unique_id']].drop_duplicates('inv_unique_id')


In [ ]:
final_injured_csv = final_injured_csv.loc[:, ['vehicle_or_pedastrian', 'injury_severity_hebrew' ,'age_group_hebrew', 'accident_timestamp', 'longitude', 'latitude']].reset_index(drop=True)


In [ ]:
final_injured_csv.columns = ['סוג נפגע', 'חומרת פגיעה', 'קבוצת גיל','תאריך ושעה' ,'longitude', 'latitude']

### remove injured not in bb

In [ ]:
injured_idx_in_poly = set()
for school_id in tqdm(poly_school_ids_final):
    center_lat, center_lon = df_total_no_idx.loc[df_total_no_idx.school_id == school_id, ['school_latitude', 'school_longitude']].values[0]
    lat_min, lon_min, lat_max, lon_max = get_bounding_box(center_lat, center_lon, 0.5)
    baseX = lon_min
    baseY = lat_min
    distanceX = lon_max
    distanceY = lat_max

    poly = Polygon([(baseX, baseY), 
                    (baseX, distanceY),
                    (distanceX, distanceY),
                    (distanceX, baseY)])
    injured = final_injured_csv.loc[:,['longitude', 'latitude']].to_dict(orient='index')
    injured_idx = [k for k, v in injured.items() if poly.contains(Point(v['longitude'], v['latitude']))]
    injured_idx_in_poly = injured_idx_in_poly.union(set(injured_idx))
    

In [ ]:
final_injured_csv = final_injured_csv.loc[list(injured_idx_in_poly)]

In [ ]:
final_injured_csv.to_csv('final_injured_csv.csv', index=False)

In [ ]:
final_injured_csv_light = final_injured_csv.loc[final_injured_csv['חומרת פגיעה'] == 'פצוע קל']

final_injured_csv_light.to_csv('final_injured_csv_light.csv', index=False)

In [ ]:
final_injured_csv_light.shape

In [ ]:
final_injured_csv_severe = final_injured_csv.loc[final_injured_csv['חומרת פגיעה'] == 'פצוע קשה']

final_injured_csv_severe.to_csv('final_injured_csv_severe.csv', index=False)

In [ ]:
final_injured_csv_severe.shape

In [ ]:
final_injured_csv_killed = final_injured_csv.loc[final_injured_csv['חומרת פגיעה'] == 'הרוג']

final_injured_csv_killed.to_csv('final_injured_csv_killed.csv', index=False)

In [ ]:
final_injured_csv_killed.shape

### Injured Statistics

In [ ]:
df_unique_injured.groupby(['age_group_hebrew', 'injury_severity_hebrew']).size().unstack(-1)

### per type and yishuv

In [ ]:
#df_unique_injured.groupby(['vehicle_or_pedastrian', 'yishuv_name']).size()

### provider code 1 by speed limit

In [ ]:
df_unique_injured_provider_code_1 = df_unique_injured.loc[df_unique_injured.provider_code == 1]

In [ ]:
df_speed_limit = df_unique_injured_provider_code_1.groupby(['speed_limit_hebrew', 'injury_severity_hebrew']).size().rename('injured_killed_count').reset_index()


In [ ]:
def calc_speed_limit_above_or_below_50(x):
    if x == 'עד 50 קמ"ש':
        return 'עד 50 קמ"ש'
    elif x == 'לא ידוע':
        return 'מגבלת מהירות לא ידועה'
    else:
        return 'מעל 50 קמ״ש'

In [ ]:
df_speed_limit['speed_limit_above_or_below_50'] = df_speed_limit['speed_limit_hebrew'].apply(calc_speed_limit_above_or_below_50)


In [ ]:
df_speed_limit['killed_severe_injured_or_light'] = df_speed_limit['injury_severity_hebrew'].apply(lambda x: x if x == 'פצוע קל' else 'פצוע קשה או הרוג')


In [ ]:
df_speed_limit_by_50 = df_speed_limit.groupby(['speed_limit_above_or_below_50', 'killed_severe_injured_or_light'])['injured_killed_count'].sum().to_frame()


In [ ]:
df_speed_limit_by_50 = df_speed_limit_by_50.unstack(-1)

In [ ]:
df_speed_limit_by_50

In [ ]:
df_speed_limit_by_50_total = df_speed_limit_by_50.sum(axis=1).rename('total_injured_killed_').to_frame()

In [ ]:
df_speed_limit_by_50 = pd.merge(left=df_speed_limit_by_50, right=df_speed_limit_by_50_total, how='left', left_index=True, right_index=True)

In [ ]:
df_speed_limit_by_50

In [ ]:
df_speed_limit_by_50['light_pct'] = 100 * df_speed_limit_by_50.iloc[:, 0] / df_speed_limit_by_50.iloc[:, 2]

In [ ]:
df_speed_limit_by_50['severe_killed_pct'] = 100 * df_speed_limit_by_50.iloc[:, 1] / df_speed_limit_by_50.iloc[:, 2]

In [ ]:
df_speed_limit_by_50

In [ ]:
df_speed_limit_by_50['injured_killed_pct'] = 

In [ ]:
import plotly.express as px

fig = px.bar(df_speed_limit_by_50.reset_index(),
             x="speed_limit_above_or_below_50",
             y="injured_killed_count",
             color="killed_severe_injured_or_light",
             color_discrete_sequence = [ "yellow" , "orange", "red"],
             title="נפגעים בגילאים 5-19 בקרבת מוסדות לימוד",
             labels={'injured_killed_count': 'נפגעים', 'date_month': 'חודש'},
             )
#fig.update_layout(xaxis=dict(tickformat="%d-%m"))

fig.show()

In [ ]:
import plotly.express as px

fig = px.bar(df_speed_limit,
             x="speed_limit_hebrew",
             y="injured_killed_count",
             color="injury_severity_hebrew",
             color_discrete_sequence = [ "yellow" , "orange", "red"],
             title="נפגעים בגילאים 5-19 בקרבת מוסדות לימוד",
             labels={'injured_killed_count': 'נפגעים', 'date_month': 'חודש'},
             )
#fig.update_layout(xaxis=dict(tickformat="%d-%m"))

fig.show()

In [ ]:
#vehicle_or_pedastrian

In [ ]:
df_unique_injured['count'] = 1

In [ ]:
import datetime

In [ ]:
df_unique_injured['date_month'] = df_unique_injured['accident_timestamp'].apply(lambda x: datetime.date(x.year, x.month, 1))


In [ ]:
table_per_month_severity = df_unique_injured.groupby(['date_month', 'injury_severity_hebrew']).size().rename('injured_killed_count').to_frame().reset_index(drop=False)


In [ ]:
table_per_month_severity.to_csv('table_per_month_severity.csv', index=False)

In [ ]:
import plotly.express as px

fig = px.bar(table_per_month_severity,
             x="date_month",
             y="injured_killed_count",
             color="injury_severity_hebrew",
             color_discrete_sequence = [ "yellow" , "orange", "red"],
             title="נפגעים בגילאים 5-19 בקרבת מוסדות לימוד",
             labels={'injured_killed_count': 'נפגעים', 'date_month': 'חודש'},
             )

fig.show()

In [ ]:
[c for c in df_unique_injured.columns if 'yishuv' in c]

In [ ]:
df_unique_injured.loc[df_unique_injured.injury_severity == 1].groupby('accident_yishuv_name').size()

In [ ]:
df_unique_injured.loc[df_unique_injured.injury_severity == 1].groupby('school_yishuv_name').size().to_frame().to_csv('yishuvs_and_killed.csv')


In [ ]:
#df_unique_injured.loc[df_unique_injured.injury_severity == 1].groupby(['school_yishuv_name', 'accident_yishuv_name']).size()


In [ ]:
#df_unique_injured.loc[(df_unique_injured.accident_month == 1)&(df_unique_injured.accident_year == 2020)&(df_unique_injured.accident_severity <=2)].drop_duplicates('inv_unique_id')


In [ ]:
#df_unique_injured.loc[(df_unique_injured.accident_month == 4)&(df_unique_injured.accident_year == 2019)&(df_unique_injured.accident_severity <=2)].drop_duplicates('inv_unique_id')


### compare months - April September January

In [ ]:
injured_killed_per_month = df_unique_injured.groupby(['accident_year', 'accident_month', 'injury_severity_hebrew', 'injury_severity']).size().rename('injured_or_killed').reset_index(drop=False)
injured_killed_per_month.to_csv('injured_killed_per_month.csv', index=False)


In [ ]:
injured_killed_per_month_covid = injured_killed_per_month.loc[(injured_killed_per_month.accident_year >= 2019)&\
                                                              (injured_killed_per_month.accident_year < 2022) &\
                                 (injured_killed_per_month.accident_month.isin([1,4,9]))].copy()


In [ ]:
months_dict = {1: 'ינואר',
               4: 'אפריל',
               9: 'ספטמבר'}

In [ ]:
injured_killed_per_month_covid['accident_month_hebrew'] = injured_killed_per_month_covid.apply(lambda x: months_dict[x['accident_month']], axis=1).values


In [ ]:
import pandas as pd
import numpy as np
import altair as alt

In [ ]:
alt.Chart(injured_killed_per_month_covid).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('accident_year:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(injured_or_killed):Q',
        axis=alt.Axis(
            grid=False,
            title=None)),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('accident_month_hebrew:N', title=None),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('injury_severity_hebrew:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['red','yellow',  'orange'],
            ),
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )

In [ ]:
inj_19_20 = injured_killed_per_month_covid.loc[injured_killed_per_month_covid.accident_year.isin([2019,2020])].copy()

inj_19_20['חומרת פציעה'] = inj_19_20['injury_severity_hebrew']

inj_19_20 = inj_19_20.sort_values('injury_severity')

alt.Chart(inj_19_20).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('accident_year:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(injured_or_killed):Q',
        axis=alt.Axis(
            grid=False,
            title=None)),

    # tell Altair which field to use to use as the set of columns to be represented in each group
    column=alt.Column('accident_month_hebrew:N', title=None),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('חומרת פציעה:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['red','yellow',  'orange'],
            ),
        ), text='sum(injured_or_killed):Q')\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )

#### 2019-2020

In [ ]:
inj_19_20 = injured_killed_per_month_covid.loc[injured_killed_per_month_covid.accident_year.isin([2019,2020])].copy()

inj_19_20['חומרת פציעה'] = inj_19_20['injury_severity_hebrew']

inj_19_20 = inj_19_20.sort_values('injury_severity')

alt.Chart(inj_19_20).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('accident_year:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(injured_or_killed):Q',
        axis=alt.Axis(
            grid=False,
            title=None)),

    # tell Altair which field to use to use as the set of columns to be represented in each group
    column=alt.Column('accident_month_hebrew:N', title=None),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('חומרת פציעה:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['red','yellow',  'orange'],
            ),
        ), text='sum(injured_or_killed):Q')\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )

#### 2020-2021

In [ ]:
inj_19_20 = injured_killed_per_month_covid.loc[injured_killed_per_month_covid.accident_year.isin([2020,2021])].copy()

inj_19_20['חומרת פציעה'] = inj_19_20['injury_severity_hebrew']

inj_19_20 = inj_19_20.sort_values('injury_severity')

alt.Chart(inj_19_20).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('accident_year:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(injured_or_killed):Q',
        axis=alt.Axis(
            grid=False,
            title=None)),

    # tell Altair which field to use to use as the set of columns to be represented in each group
    column=alt.Column('accident_month_hebrew:N', title=None),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('חומרת פציעה:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['red','yellow',  'orange'],
            ),
        ), text='sum(injured_or_killed):Q')\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )

In [ ]:
import plotly.express as px

fig = px.bar(table_per_month_severity,
             x="date_month",
             y="injured_killed_count",
             color="injury_severity_hebrew",
             color_discrete_sequence = [ "yellow" , "orange", "red"],
             title="נפגעים בגילאים 5-19 בקרבת מוסדות לימוד",
             labels={'injured_killed_count': 'נפגעים', 'date_month': 'חודש'},
             )
#fig.update_layout(xaxis=dict(tickformat="%d-%m"))

fig.show()

In [ ]:
fig.write_html("injured_per_month_severity.html")

### create jsons

#### get all schools

In [ ]:
schools_data = json.load(open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/schools_names.json"))


In [ ]:
all_schools_df = pd.DataFrame(schools_data)

In [ ]:
all_schools_ids = all_schools_df.school_id.unique()

#### injured_around_schools_api_2022_json

In [ ]:
df_total_injured_schools_per_year = (
    df.groupby(
        [
            "school_id",
            "accident_year",   
            "injury_severity"
        ]
    )
    .size()
    .reset_index(name="injured_count")
    .loc[
        :,
        [
            "school_id",
            "accident_year",
            "injury_severity",
            "injured_count",
        ],
    ]
)
df_total_injured_schools_per_year = df_total_injured_schools_per_year.set_index(
    [
            "school_id",
            "accident_year",   
            "injury_severity"

    ]
).unstack(-1)
df_total_injured_schools_per_year.fillna({"injured_count": 0, "total_injured_count": 0}, inplace=True)

df_total_injured_schools_per_year.loc[:, (slice("injured_count"), slice(None))] = df_total_injured_schools_per_year.loc[
    :, (slice("injured_count"), slice(None))
].apply(lambda x: x.apply(int))

df_total_injured_schools_per_year["total_injured_count"] = (
    df_total_injured_schools_per_year.loc[:, ["injured_count"]].sum(axis=1)
).apply(int)

df_total_injured_schools_per_year.columns = ['killed_count', 'severly_injured_count', 'light_injured_count', 'total_injured_killed_count']


df_total_injured_schools_per_year = df_total_injured_schools_per_year.reset_index(drop=False).set_index('school_id')

injured_schools_per_year_list = df_total_injured_schools_per_year.reset_index().to_dict(orient='records')

school_ids = df_total_injured_schools_per_year.index.unique()

total_dictionary_for_json_api = {}
for school_id in all_schools_ids:
    total_dictionary_for_json_api[str(school_id)] = [r for r in injured_schools_per_year_list if r["school_id"] == school_id]
with open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/injured_around_schools_api_2022.json", "w") as f:
    json.dump(total_dictionary_for_json_api, f)

#### injured_around_months_graph_data_api_2022_json

In [ ]:
df_total_injured_schools_per_month = (
    df.groupby(
        [
            "school_id",
            "accident_month_hebrew",   
        ]
    )
    .size()
    .reset_index(name="count_1")
    .loc[
        :,
        [
            "school_id",
            "accident_month_hebrew",
            "count_1",
        ],
    ]
)
injured_schools_per_month_list = df_total_injured_schools_per_month.to_dict(orient='records')
total_dictionary_for_json_api = {}
for school_id in all_schools_ids:
    total_dictionary_for_json_api[str(school_id)] = [r for r in injured_schools_per_month_list if r["school_id"] == school_id]
with open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/injured_around_schools_months_graphs_data_api_2022.json", "w") as f:
    json.dump(total_dictionary_for_json_api, f)


#### injured_around_sex_graph_data_api_2022_json


In [ ]:
df_total_injured_schools_per_sex = (
    df.groupby(
        [
            "school_id",
            "sex_hebrew",   
        ]
    )
    .size()
    .reset_index(name="count_1")
    .loc[
        :,
        [
            "school_id",
            "sex_hebrew",
            "count_1",
        ],
    ]
)
injured_schools_per_sex_list = df_total_injured_schools_per_sex.to_dict(orient='records')
total_dictionary_for_json_api = {}
for school_id in all_schools_ids:
    total_dictionary_for_json_api[str(school_id)] = [r for r in injured_schools_per_sex_list if r["school_id"] == school_id]
with open("/Users/atalya/Documents/anyway_main/repos/anyway/static/data/schools/injured_around_schools_sex_graphs_data_api_2022.json", "w") as f:
    json.dump(total_dictionary_for_json_api, f)


### calculate ncr

In [ ]:
import operator as op
from functools import reduce

def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom  # or / in Python 2

In [ ]:
ncr(50,3)

In [ ]:
ncr(100,3)

In [ ]:
ncr(193,3)